In [1]:
import networkx as nx
import markov_clustering as mc
from networkx.algorithms.community import fast_label_propagation_communities
import random
import numpy as np 
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import math
import pickle

In [2]:
g = nx.read_weighted_edgelist('4932.protein.links.v12.0.txt',comments = "#",nodetype=str)

# remove some edges, guess we're doing 700 now 
for u, v in g.edges:
    if g.get_edge_data(u, v)['weight'] < 700:
      g.remove_edge(u, v)

#remove weights
for node, edges in nx.to_dict_of_dicts(g).items():
    for edge, attrs in edges.items():
        attrs.pop('weight', None)

matrix = nx.to_numpy_array(g)
node_list = list(g.nodes)
related_proteins = ['4932.YMR190C','4932.YNL088W','4932.YLR234W','4932.YPL024W','4932.YMR167W' ]

FileNotFoundError: [Errno 2] No such file or directory: '4932.protein.links.v12.0.txt'

In [ ]:
def mcl(mtx,inflation_parameter):
    result = mc.run_mcl(mtx,inflation = inflation_parameter)
    clusters = mc.get_clusters(result)

    #relabelling node names 
    for i in range(0,len(clusters)):
        clu_list = list(clusters[i])
        
        for j in range(0,len(clu_list)):
            name = node_list[clu_list[j]]
            clu_list[j] = name
        clusters[i] = tuple(clu_list)

    return inflation_parameter, result, clusters

def similarity_of_clusterings(clusters1,name1, clusters2,name2, num):
    clusters1 = sorted(clusters1,key = len, reverse = True)
    clusters2 = sorted(clusters2,key = len, reverse = True)

    sim_dict = {}
    for i in range(0,len(clusters1)) :
        similarity = []
        for j in range(0,len(clusters2)):
            counter = 0
            for k in clusters1[i]:
                if k in clusters2[j]:
                    counter +=1
                    
            # bool = 0: shit sim meas, bool = 1: jaccards
            if num == 0:
                score = 2*counter/(len(clusters1[i])+len(clusters2[j]))
            elif num == 1:
                score = counter/(len(clusters1[i])+len(clusters2[j])-counter)
                
            if score > 0:
                similarity.append((str(name2)+str(j),len(clusters2[j]),score))
        similarity = sorted(similarity, key = lambda x:x[-1], reverse= True)
        sim_dict[(str(name1)+str(i),len(clusters1[i]))] = similarity
    return sim_dict

def large_comm(cluster,threshold):
    large = []
    for i in cluster:
        if len(i)>=threshold:
            large.append(i)

    return large

def similarity_of_algo(dict):
    score_list=[]
    for i in dict: 
        score_list.append(i[1]*dict[i][0][-1])
    score = sum(score_list)/6538
    
    return score, score_list

In [ ]:
with open('markov_clustering_14.pkl', 'rb') as file:
    markov_clustering_14 = pickle.load(file)
clusters = markov_clustering_14[2]

with open('Fast_PP_1.pkl', 'rb') as file:
    label_prop_1 = pickle.load(file)
label_prop_1 = sorted(label_prop_1, key=len,reverse = True)

In [ ]:
similarity_of_clusterings(large_comm(label_prop_1,30),'lp',large_comm(clusters,30),'mc',0)